In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from braindecode.models import EEGConformer
import pandas as pd
from collections import OrderedDict
import math
import importlib
import Spatial_attention_FACED
importlib.reload(Spatial_attention_FACED)
from Spatial_attention_FACED import ShallowAttentionNet
from shallow_laurits import ShallowFBCSPNet
from torchinfo import summary
from weight_init import init_weights

In [ ]:
in_chans = 32
n_classes = 3
input_window_samples = 400
model = ShallowAttentionNet(in_chans,n_classes,input_window_samples)
model.apply(init_weights)

ShallowAttentionNet(
  (temporal): Conv2d(1, 40, kernel_size=(1, 25), stride=(1, 1))
  (spatial_att): SpatialAttention(
    (conv): Conv2d(40, 40, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (pool): MaxPool2d(kernel_size=(1, 5), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
  )
  (batch_norm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 50), stride=(1, 50), padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1280, out_features=4, bias=True)
)

In [ ]:
print(model)

ShallowAttentionNet(
  (temporal): Conv2d(1, 40, kernel_size=(1, 25), stride=(1, 1))
  (spatial_att): SpatialAttention(
    (conv): Conv2d(40, 40, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (pool): MaxPool2d(kernel_size=(1, 5), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
  )
  (batch_norm): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): AvgPool2d(kernel_size=(1, 50), stride=(1, 50), padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=1280, out_features=4, bias=True)
)


In [ ]:
summary(model,input_size=(2, 32, 400),col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape
ShallowAttentionNet                      [2, 32, 400]              [2, 4]                    --                        --
├─Conv2d: 1-1                            [2, 1, 32, 400]           [2, 40, 32, 376]          1,040                     [1, 25]
├─SpatialAttention: 1-2                  [2, 40, 32, 376]          [2, 40, 32, 75]           --                        --
│    └─Conv2d: 2-1                       [2, 40, 32, 376]          [2, 40, 32, 376]          1,600                     [1, 1]
│    └─MaxPool2d: 2-2                    [2, 40, 32, 376]          [2, 40, 32, 75]           --                        [1, 5]
├─BatchNorm2d: 1-3                       [2, 40, 32, 75]           [2, 40, 32, 75]           80                        --
├─AvgPool2d: 1-4                         [2, 40, 32, 75]           [2, 40, 32, 1]            --                        [1, 